In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#INFO:序列特征等长度不一致，需要补全。比不规则张量更灵活。

#遮盖的作用是告知序列处理层输入中有某些时间步骤丢失，因此在处理数据时应将其跳过。
#填充是遮盖的一种特殊形式，其中被遮盖的步骤位于序列的起点或开头。
#   填充是出于将序列数据编码成连续批次的需要：为了使批次中的所有序列适合给定的标准长度，有必要填充或截断某些序列。

In [2]:
### 2.11.1 填充序列数据
#在处理序列数据时，各个样本常常具有不同长度。请考虑以下示例（文本被切分为单词）：
[
  ["Hello", "world", "!"],
  ["How", "are", "you", "doing", "today"],
  ["The", "weather", "will", "be", "nice", "tomorrow"],
]

[['Hello', 'world', '!'],
 ['How', 'are', 'you', 'doing', 'today'],
 ['The', 'weather', 'will', 'be', 'nice', 'tomorrow']]

In [4]:
#进行词汇查询后，数据可能会被向量化为整数，例如：
[
  [71, 1331, 4231],
  [73, 8, 3215, 55, 927],
  [83, 91, 1, 645, 1253, 927],
]

[[71, 1331, 4231], [73, 8, 3215, 55, 927], [83, 91, 1, 645, 1253, 927]]

In [5]:
#此数据是一个嵌套列表，其中各个样本的长度分别为 3、5 和 6。由于深度学习模型的输入数据必须为单一张量
#   （例如在此例中形状为 (batch_size, 6, vocab_size)），短于最长条目的样本需要用占位符值进行填充
#   （或者，也可以在填充短样本前截断长样本）。

#Keras 提供了一个效用函数来截断和填充 Python 列表，
#   使其具有相同长度：tf.keras.preprocessing.sequence.pad_sequences。
raw_inputs = [
    [711, 632, 71],
    [73, 8, 3215, 55, 927],
    [83, 91, 1, 645, 1253, 927],
]

# By default, this will pad using 0s; it is configurable via the
# "value" parameter.
# Note that you could "pre" padding (at the beginning) or
# "post" padding (at the end).
# We recommend using "post" padding when working with RNN layers
# (in order to be able to use the
# CuDNN implementation of the layers).
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(
    raw_inputs, padding="post"
)
padded_inputs


array([[ 711,  632,   71,    0,    0,    0],
       [  73,    8, 3215,   55,  927,    0],
       [  83,   91,    1,  645, 1253,  927]])

In [ ]:
### 2.11.2 遮盖
#既然所有样本现在都具有了统一长度，那就必须告知模型，数据的某些部分实际上是填充，应该忽略。这种机制就是遮盖。

#在 Keras 模型中引入输入掩码有三种方式：
#   添加一个 keras.layers.Masking 层。
#   使用 mask_zero=True 配置一个 keras.layers.Embedding 层。
#   在调用支持 mask 参数的层（如 RNN 层）时，手动传递此参数。


In [6]:
### 2.11.3 掩码生成层：Embedding 和 Masking
#这些层将在后台创建一个掩码张量（形状为 (batch, sequence_length) 的二维张量），
#   并将其附加到由 Masking 或 Embedding 层返回的张量输出上。

embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
masked_output = embedding(padded_inputs)
masked_output._keras_mask



<tf.Tensor: shape=(3, 6), dtype=bool, numpy=
array([[ True,  True,  True, False, False, False],
       [ True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True,  True]])>

In [16]:
masked_output.shape

TensorShape([3, 6, 16])

In [12]:
masking_layer = layers.Masking()
# Simulate the embedding lookup by expanding the 2D input to 3D,
# with embedding dimension of 10.
unmasked_embedding = tf.cast(
    tf.tile(tf.expand_dims(padded_inputs, axis=-1), [1, 1, 10]), tf.float32
)
masked_embedding = masking_layer(unmasked_embedding)
masked_embedding._keras_mask #您可以在输出结果中看到，该掩码是一个形状为 (batch_size, sequence_length) 的二维布尔张量，
                             #  其中每个 False 条目表示对应的时间步骤应在处理时忽略。

<tf.Tensor: shape=(3, 6), dtype=bool, numpy=
array([[ True,  True,  True, False, False, False],
       [ True,  True,  True,  True,  True, False],
       [ True,  True,  True,  True,  True,  True]])>

In [14]:
unmasked_embedding.shape

TensorShape([3, 6, 10])

In [15]:
masked_embedding.shape

TensorShape([3, 6, 10])

In [18]:
### 2.11.4 函数式 API 和序列式 API 中的掩码传播
#在使用函数式 API 或序列式 API 时，由 Embedding 或 Masking 层生成的掩码将通过网络传播给任何能够使用它们的层（如 RNN 层）。
#Keras 将自动提取与输入相对应的掩码，并将其传递给任何知道该掩码使用方法的层。

#例如，在下面的序贯模型中，LSTM 层将自动接收掩码，这意味着它将忽略填充的值：
model = keras.Sequential(
    [layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True), layers.LSTM(32),]
)


In [19]:
#对以下函数式 API 的情况也是如此：
inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

model = keras.Model(inputs, outputs)

In [ ]:
### 2.11.5 更多功能
#将掩码张量直接传递给层、在自定义层中支持遮盖、在兼容层上选择启用掩码传播更多功能，
#可参考：https://tensorflow.google.cn/guide/keras/masking_and_padding?hl=zh-cn#%E5%B0%86%E6%8E%A9%E7%A0%81%E5%BC%A0%E9%87%8F%E7%9B%B4%E6%8E%A5%E4%BC%A0%E9%80%92%E7%BB%99%E5%B1%82

In [ ]:
### 2.11.6 tf.data的padding
#dataset.padded_batch 
#可参考6.1.3节